In [ ]:
import numpy as np
import pandas as pd
import scipy.stats
import matplotlib.pyplot as plt
from tqdm.notebook import trange, tqdm

In [ ]:
replicas = 100
h = 5000
lambda_monto_reclamo = 1
media_reclamo = 1/lambda_monto_reclamo
lambda_tiempo_interreclamo = 2
media_tiempo = 1/lambda_tiempo_interreclamo
c = 3
us = np.linspace(1,2,3,4,5)

#Montecarlo Condicionado

In [ ]:
#Generar tiempos entre reclamos
def generarTiempos(media_tiempo):
    counter = 0
    T = [0]
    T_acumulado = [0]
    while counter<h:
        tiempo = np.random.exponential(media_tiempo)
        if counter + tiempo > 5000:
            break
        T.append(tiempo)
        counter += tiempo
        T_acumulado.append(counter)
    return T, T_acumulado

In [ ]:
#Generar reclamos
def generarReclamos(T, media_reclamo):
    reclamos = [0]
    for i in range(len(T)-1):
        reclamo = np.random.exponential(media_reclamo)
        reclamos.append(reclamo)
    return reclamos

In [ ]:
def getReservas(T, reclamos, u, c):
    #Proceso de reserva
    #U(t)=u+cT-sum(j=1 a k de Xj a la i)
    reservas = []
    for i in range(len(T)):
        if i == 0:
            reserva = u + c*T_acumulado[i]-reclamos[i]
        else:
            reserva = reservas[i-1] + c*T[i]-reclamos[i]
        reservas.append(reserva)
        #if reservas[-1] < 0:
        #    break
    return reservas

In [ ]:
import numpy as np
import scipy.stats


def intervalo_confianza(ruinas):
    inter = 0.95
    ruinas = 1.0 * np.array(ruinas)
    n = len(ruinas)
    m, se = np.mean(ruinas), scipy.stats.sem(ruinas)
    h = se * scipy.stats.t.ppf((1 + inter) / 2., n-1)
    return m, m-h, m+h

In [ ]:
import statsmodels.stats.api as sms

sms.DescrStatsW(a).tconfint_mean()

In [ ]:
from statistics import NormalDist

def confidence_interval(data, confidence=0.95):
  dist = NormalDist.from_samples(data)
  z = NormalDist().inv_cdf((1 + confidence) / 2.)
  h = dist.stdev * z / ((len(data) - 1) ** .5)
  return dist.mean - h, dist.mean + h

In [ ]:
numCasosRuina = []
probRuina = []
tiempoPromedioRuina = []
tiemposRuinas = []


for u in trange(len(us)):
  ruinas = 0
  tiemposRuina = []
  I = 0
  V = 0
  ind = 0
  var = 0
  for i in trange(replicas):
    T, T_acumulado = generarTiempos(media_tiempo)
    reclamos = generarReclamos(T, media_reclamo)
    reservas = []
    for t in range(len(T)):
      if t == 0:
        reserva = u + c*T_acumulado[t]-reclamos[i]
      else:
        reserva = reservas[t-1] + c*T[t]-reclamos[i]
      reservas.append(reserva)
      if reserva < 0:
        print('quiebra')
        ind += 1
        break
    if reservas[-1]<0:
      tiemposRuina.append(T_acumulado[len(reservas)-1])
      ruinas += 1
  tiemposRuinas.append(tiemposRuina)
  tiempoPromedioRuina.append(np.mean(tiemposRuina))
  numCasosRuina.append(ruinas)
  probRuina.append(ruinas/replicas)

In [ ]:
reservas